In [2]:
import sys
sys.path.append('/n/home09/kboone/software/CMBL_x_WL/Half_Data/')

import matplotlib.pyplot as plt
from Bin_Helpers import *
import numpy as np
import os

import matplotlib
matplotlib.style.use('des_dr1')

In [3]:
survey = 'Planck'

nside = 1024
lmin = 2
lmax = 3072 # Must be 3072 or less
data_len = lmax-lmin
numBins = 15

fid_dir = '/n/netscratch/dvorkin_lab/Lab/kboone/cmb_conv/cls/'
deriv_dir = fid_dir

all_files = np.array(os.listdir(fid_dir))
fid_inds = [i for i, s in enumerate(all_files) if s.startswith('f')]
fid_files = all_files[fid_inds]

all_files = np.array(os.listdir(deriv_dir))
Om_inds = [i for i, s in enumerate(all_files) if s.startswith('O')]
s8_inds = [i for i, s in enumerate(all_files) if s.startswith('s')]
Om_files = all_files[Om_inds]
s8_files = all_files[s8_inds]
Om_files = np.array([name for name in Om_files if 'f' not in name])
s8_files = np.array([name for name in s8_files if 'f' not in name])

fid = np.zeros((len(fid_files), data_len), dtype = np.float64)
for i in np.arange(len(fid)):
    fid[i] = np.load(fid_dir + fid_files[i], allow_pickle=True).item()[survey][lmin:lmax]

ave_fid = np.average(fid, axis=0)
std_fid = np.std(fid, axis=0)
fid = (fid-ave_fid) / std_fid

C = np.atleast_2d(np.cov(fid.T, bias=True))
max_C = np.max(C)
C /= max_C

partial_Om = np.zeros((len(Om_files), data_len), dtype = np.float64)
for i in np.arange(len(partial_Om)):
    partial_Om[i] = np.load(deriv_dir + Om_files[i], allow_pickle=True).item()[survey][lmin:lmax]

partial_Om = partial_Om / std_fid
partial_Om = np.atleast_2d(np.average(partial_Om, axis = 0))

partial_s8 = np.zeros((len(s8_files), data_len), dtype = np.float64)
for i in np.arange(len(partial_s8)):
    partial_s8[i] = np.load(deriv_dir + s8_files[i], allow_pickle=True).item()[survey][lmin:lmax]

partial_s8 = partial_s8 / std_fid
partial_s8 = np.atleast_2d(np.average(partial_s8, axis = 0))

D = np.append(partial_Om, partial_s8, axis = 0).T
D /= np.sqrt(max_C)

In [6]:
xlen = 100
ylen = 100

x_test = np.linspace(0, data_len-1, xlen)
y_test = np.linspace(0, numBins-1, ylen)

x_grid, y_grid = np.meshgrid(x_test, y_test, indexing='ij')
x_flat = x_grid.flatten()
y_flat = y_grid.flatten()

det_flat = []
for i in np.arange(len(x_flat)):
    if i % 1000 == 0:
        print(i)
    if y_flat[i] >= x_flat[i]:
        det_flat.append(0)
    elif (numBins - y_flat[i]) >= (data_len - x_flat[i]):
        det_flat.append(0)
    else:
        try:
            det_flat.append(det_fish(create_linlin_matrix(data_len, numBins, 
                                                          points=[[x_flat[i], y_flat[i]]]),
                                     C, D))
        except:
            det_flat.append(0)
        
det_flat = np.array(det_flat)
det_grid = np.reshape(det_flat, (xlen, ylen))
det_grid = det_grid / np.max(det_grid)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [11]:
fig, ax = plt.subplots(dpi=120)

p = ax.pcolormesh(
    x_grid.T, y_grid.T, det_grid.T,
    shading='nearest',          # avoids seams
    vmin=0, vmax=1,
    edgecolors='none',
    antialiased=False,
    rasterized=True             # rasterize the mesh only
)

fig.colorbar(p, ax=ax)
ax.set_title(r'Fisher Determinant for Planck $\kappa_\text{CMB}$ Auto $C_\ell$' + "'s", fontsize=24, y=1.02)
ax.set_xlabel(r'$\ell$ Position of Pivot', fontsize=24)
ax.set_ylabel('Bin Position of Pivot', fontsize=24)

# ax.invert_yaxis()

# plt.show()

fig.savefig('../Paper_Plots/Qual_Heatmap.pdf',
            bbox_inches='tight', format='pdf', dpi=300)

plt.close(fig)

In [4]:
plt.figure(dpi = 120)
p = plt.pcolormesh(x_grid.T, y_grid.T, det_grid.T, shading='auto',
                  vmax=1, vmin=0)
plt.colorbar(p)
plt.title(r'Normalized Fisher Determinant for Planck Auto $C_\ell$'+'\'s')
plt.xlabel(r'$\ell$ Position of Pivot')
plt.ylabel('Bin Position of Pivot')
# plt.gca().invert_yaxis()
plt.savefig('../Paper_Plots/Flip_Planck_Heatmap.pdf', bbox_inches='tight', format='pdf')
plt.close()
# plt.show()